# Advanced processing of Strands Agents Response

Strands Agents allows you to intercept and process events as they happen during agent execution using two methods: 
    
- **Async iterators**: ideal for asynchronous frameworks like FastAPI, aiohttp, or Django Channels. For these environments, the SDK offers the `stream_async` method which returns an asynchronous iterator. 
- **Callback handlers**: allow you to intercept and process events as they happen during agent execution. This enables real-time monitoring, custom output formatting, and integration with external systems.

In this example, we will show you how to use both methods to handle calls on your agent


## Agent Details
<div style="float: left; margin-right: 20px; ">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |async iterators, callback handlers                 |
|Agent Structure     |single agent architecture                          |
|Native tools used   |calculator                                         |
|Custom tools created|Weather forecast                                   |

</div>

## Architecture

<div style="text-align:left;">
    <img src="images/architecture.png" width="65%" />
</div>

## Key Features
* Async Iterators for Streaming
* Callback Handlers


## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock

Let's now install the requirement packages for our Strands Agent Agent

In [9]:
# installing pre-requisites
!pip install -r requirements.txt

  Attempting uninstall: starlette
    Found existing installation: starlette 0.50.0
    Uninstalling starlette-0.50.0:
      Successfully uninstalled starlette-0.50.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.22.0 requires packaging<25, but you have packaging 25.0 which is incompatible.
mlflow 2.22.0 requires pyarrow<20,>=4.0.0, but you have pyarrow 22.0.0 which is incompatible.
mlflow-skinny 2.22.0 requires packaging<25, but you have packaging 25.0 which is incompatible.
sagemaker 2.245.0 requires attrs<24,>=23.1.0, but you have attrs 25.4.0 which is incompatible.
sagemaker 2.245.0 requires importlib-metadata<7.0,>=1.4.0, but you have importlib-metadata 8.7.0 which is incompatible.
sagemaker 2.245.0 requires numpy==1.26.4, but you have numpy 2.3.5 which is incompatible.
sagemaker 2.245.0 requires packaging<25,>=23.0, but you have packaging 25.0 

### Importing dependency packages

Now let's import the dependency packages

In [11]:
import asyncio

import httpx
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
from strands import Agent, tool
from strands_tools import calculator

## Method 1 - Async Iterators for Streaming


Strands Agents provides support for asynchronous iterators through the `stream_async` method, enabling real-time streaming of agent responses in asynchronous environments like web servers, APIs, and other async applications.

Since we are show casing this example in a notebook, we need to apply `nest_asyncio` to allow nested use of `asyncio.run` and `loop.run_until_complete`

In [12]:
nest_asyncio.apply()

### Creating and invoking agent with stream_async

Let's now create our agent with a built-in calculator tool and no `callback_handler`. We will use the `stream_async` method to iteract over the streamed agent events

In [13]:
# Initialize our agent without a callback handler
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    tools=[calculator], 
    callback_handler=None)

# Async function that iterators over streamed agent events


async def process_streaming_response():
    agent_stream = agent.stream_async("Calculate 2+2")
    async for event in agent_stream:
        print(event)


# Run the agent
asyncio.run(process_streaming_response())

{'init_event_loop': True}
{'start': True}
{'start_event_loop': True}
{'event': {'messageStart': {'role': 'assistant'}}}
{'event': {'contentBlockDelta': {'delta': {'text': 'I can'}, 'contentBlockIndex': 0}}}
{'data': 'I can', 'delta': {'text': 'I can'}, 'agent': <strands.agent.agent.Agent object at 0x7f2b03011070>, 'event_loop_cycle_id': UUID('eaa33311-5937-4031-8c83-4d61efab67ff'), 'request_state': {}, 'event_loop_cycle_trace': <strands.telemetry.metrics.Trace object at 0x7f2b02e54290>, 'event_loop_cycle_span': NonRecordingSpan(SpanContext(trace_id=0x00000000000000000000000000000000, span_id=0x0000000000000000, trace_flags=0x00, trace_state=[], is_remote=False))}
{'event': {'contentBlockDelta': {'delta': {'text': ' calculate'}, 'contentBlockIndex': 0}}}
{'data': ' calculate', 'delta': {'text': ' calculate'}, 'agent': <strands.agent.agent.Agent object at 0x7f2b03011070>, 'event_loop_cycle_id': UUID('eaa33311-5937-4031-8c83-4d61efab67ff'), 'request_state': {}, 'event_loop_cycle_trace': <

╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 2+2                 │                                                                            │
│  │ Result    │ 4                   │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'message': {'role': 'user', 'content': [{'toolResult': {'status': 'success', 'content': [{'text': 'Result: 4'}], 'toolUseId': 'tooluse_MxhqwsvlRdms9NqOHpvwAQ'}}]}}
{'start': True}
{'start': True}
{'start_event_loop': True}
{'event': {'messageStart': {'role': 'assistant'}}}
{'event': {'contentBlockDelta': {'delta': {'text': 'The result'}, 'contentBlockIndex': 0}}}
{'data': 'The result', 'delta': {'text': 'The result'}, 'agent': <strands.agent.agent.Agent object at 0x7f2b03011070>, 'event_loop_cycle_id': UUID('9093f269-2d85-40d8-ba13-be4e7fdd9547'), 'request_state': {}, 'event_loop_cycle_trace': <strands.telemetry.metrics.Trace object at 0x7f2b08885f70>, 'event_loop_cycle_span': NonRecordingSpan(SpanContext(trace_id=0x00000000000000000000000000000000, span_id=0x0000000000000000, trace_flags=0x00, trace_state=[], is_remote=False)), 'model': <strands.models.bedrock.BedrockModel object at 0x7f2b02c1d2b0>, 'system_prompt': None, 'messages': [{'role': 'user', 'content': [{'text': 'Calculate 

###  Tracking event loop lifecycle

This example illustrates the event loop lifecycle and how events relate to each other. It's useful for understanding the flow of execution in the Strands Agent:

Let's create some printing format code to better analyse the agent stream events. We will continue to use the same agent for it

In [14]:
# Async function that iterators over streamed agent events


async def process_streaming_response():
    agent_stream = agent.stream_async("What is the capital of France and what is 42+7?")
    async for event in agent_stream:
        # Track event loop lifecycle
        if event.get("init_event_loop", False):
            print("🔄 Event loop initialized")
        elif event.get("start_event_loop", False):
            print("▶️ Event loop cycle starting")
        elif event.get("start", False):
            print("📝 New cycle started")
        elif "message" in event:
            print(f"📬 New message created: {event['message']['role']}")
        elif event.get("force_stop", False):
            print(
                f"🛑 Event loop force-stopped: {event.get('force_stop_reason', 'unknown reason')}"
            )

        # Track tool usage
        if "current_tool_use" in event and event["current_tool_use"].get("name"):
            tool_name = event["current_tool_use"]["name"]
            print(f"🔧 Using tool: {tool_name}")

        # Show only a snippet of text to keep output clean
        if "data" in event:
            # Only show first 20 chars of each chunk for demo purposes
            data_snippet = event["data"][:20] + (
                "..." if len(event["data"]) > 20 else ""
            )
            print(f"📟 Text: {data_snippet}")

    return event["result"]


# Run the agent
asyncio.run(process_streaming_response())

🔄 Event loop initialized
📝 New cycle started
▶️ Event loop cycle starting
📟 Text: I
📟 Text:  can
📟 Text:  answer
📟 Text:  both parts
📟 Text:  of your question.
📟 Text:  For
📟 Text:  the capital of Fran...
📟 Text: , it's Paris
📟 Text: . For the
📟 Text:  calculation
📟 Text:  part
📟 Text: , I
📟 Text: 'll use the calculat...
🔧 Using tool: calculator
🔧 Using tool: calculator
🔧 Using tool: calculator
🔧 Using tool: calculator
📬 New message created: assistant


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 42+7                │                                                                            │
│  │ Result    │ 49                  │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

📬 New message created: user
📝 New cycle started
📝 New cycle started
▶️ Event loop cycle starting
📟 Text: The capital of
📟 Text:  France is Paris,
📟 Text:  and the
📟 Text:  result of 42
📟 Text: +7 is
📟 Text:  49.
📬 New message created: assistant


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The capital of France is Paris, and the result of 42+7 is 49.'}]}, metrics=EventLoopMetrics(cycle_count=4, tool_metrics={'calculator': ToolMetrics(tool={'toolUseId': 'tooluse_RogIjaW5Q7KIpBArj9IBeQ', 'name': 'calculator', 'input': {'expression': '42+7'}}, call_count=2, success_count=2, error_count=0, total_time=0.010325431823730469)}, cycle_durations=[1.09857177734375, 0.9177510738372803], traces=[<strands.telemetry.metrics.Trace object at 0x7f2b02e54290>, <strands.telemetry.metrics.Trace object at 0x7f2b08885f70>, <strands.telemetry.metrics.Trace object at 0x7f2b02e831d0>, <strands.telemetry.metrics.Trace object at 0x7f2b08d66390>], accumulated_usage={'inputTokens': 8194, 'outputTokens': 190, 'totalTokens': 8384}, accumulated_metrics={'latencyMs': 5620}), state={})

### FastAPI Integration

You can also integrate your `stream_async` with FastAPI to create a streaming endpoint to your applications. For this, we will add a `weather_forecast` tool to our agent. The architecture update looks as following

<div style="text-align:left;">
    <img src="images/architecture_2.png" width="65%" />
</div>

In [15]:
# Tool definition


@tool
def weather_forecast(city: str, days: int = 3) -> str:
    return f"Weather forecast for {city} for the next {days} days..."


# FastAPI app
app = FastAPI()


class PromptRequest(BaseModel):
    prompt: str


@app.post("/stream")
async def stream_response(request: PromptRequest):
    async def generate():
        agent = Agent(tools=[calculator, weather_forecast], callback_handler=None)
        try:
            async for event in agent.stream_async(request.prompt):
                if "data" in event:
                    yield event["data"]
        except Exception as e:
            yield f"Error: {str(e)}"

    return StreamingResponse(generate(), media_type="text/plain")


# Function to start server without blocking


async def start_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8001, log_level="info")
    server = uvicorn.Server(config)
    await server.serve()


# Run server as background task
if "server_task" not in globals():
    server_task = asyncio.create_task(start_server())
    await asyncio.sleep(0.1)  # Give server time to start

print("✅ Server is running at http://0.0.0.0:8001")

✅ Server is running at http://0.0.0.0:8001


#### Invoking the FastAPI agent
And we can now invoke the agent with a prompt 

In [16]:
async def fetch_stream():
    async with httpx.AsyncClient() as client:
        async with client.stream(
            "POST",
            "http://0.0.0.0:8001/stream",
            json={"prompt": "What is weather in NYC?"},
        ) as response:
            async for line in response.aiter_lines():
                if line.strip():  # Skip empty lines
                    print("Received:", line)


await fetch_stream()

Received: I'll check the current weather forecast for New York City for you.Here is the weather forecast for New York City (NYC) for the next 3 days. The forecast provides you with information about expected conditions, temperatures, and other relevant weather details for the upcoming days.
Received: Is there any specific weather information you're particularly interested in, or would you like to check the forecast for a different number of days?


## Method 2 - Callback Handlers for streaming

Callback handlers are a powerful feature of the Strands Agents that allow you to intercept and process events as they happen during agent execution. This enables real-time monitoring, custom output formatting, and integration with external systems.



Callback handlers receive events in real-time as they occur during an agent's lifecycle:

- Text generation from the model
- Tool selection and execution
- Reasoning process
- Errors and completions


Let's now create a custom callback handler function that formats the event inputs to highlight tool usage and model output. To do so, we will again use the agent with a calculator tool only

<div style="text-align:left;">
    <img src="images/architecture.png" width="65%" />
</div>

In [18]:
def custom_callback_handler(**kwargs):
    # Process stream data
    if "data" in kwargs:
        print(f"MODEL OUTPUT: {kwargs['data']}")
    elif "current_tool_use" in kwargs and kwargs["current_tool_use"].get("name"):
        print(f"\nUSING TOOL: {kwargs['current_tool_use']['name']}")


# Create an agent with custom callback handler
agent = Agent(   
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
        tools=[calculator], 
        callback_handler=custom_callback_handler)

agent("Calculate 2+5")

MODEL OUTPUT: I'll calculate 
MODEL OUTPUT: 2+5 
MODEL OUTPUT: for you.

USING TOOL: calculator

USING TOOL: calculator

USING TOOL: calculator

USING TOOL: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 2+5                 │                                                                            │
│  │ Result    │ 7                   │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

MODEL OUTPUT: The result of
MODEL OUTPUT:  2+5
MODEL OUTPUT:  is 7
MODEL OUTPUT: .


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The result of 2+5 is 7.'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'calculator': ToolMetrics(tool={'toolUseId': 'tooluse_-aSFSQKmSFWLAkKnP8HClg', 'name': 'calculator', 'input': {'expression': '2+5'}}, call_count=1, success_count=1, error_count=0, total_time=0.004578828811645508)}, cycle_durations=[0.7550868988037109], traces=[<strands.telemetry.metrics.Trace object at 0x7f2b02e82e70>, <strands.telemetry.metrics.Trace object at 0x7f2b009cb560>], accumulated_usage={'inputTokens': 3974, 'outputTokens': 81, 'totalTokens': 4055}, accumulated_metrics={'latencyMs': 2024}), state={})

### Congratulations!

In this notebook you learned how to stream your agents outputs using async iteractors and callback handlers. 